In [1]:
from __future__ import print_function
import os, sys
import time
import math
import argparse
from pathlib import Path
import torch
from utils.data_load import *
import ResNet as models
from config.settings import *
from utils.data_preprocess import *
from torch.utils.tensorboard import SummaryWriter


print(torch.cuda.is_available())
torch.cuda.empty_cache()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

True


In [2]:
current_path = os.getcwd()
root_path= '/'.join(current_path.split('/')[:current_path.split('/').index('Code')])
datasets_path=os.path.join(root_path,'DataSets')

# Training settings
st = Settings()
options = st.get_options()
no_cuda =False
seed = options['seed']
cuda = not no_cuda and torch.cuda.is_available()

In [ ]:
# miccai train, isbi infer
'''options['experiment'] = 'resnet_DAN_full_miccai_train2'
model1_path = '76_model.pth'
model2_path = '7_model.pth
options['test_folder'] = os.path.join(datasets_path, 'ISBI/train/')''''

In [3]:
# isbi train, isbi infer
options['experiment'] = 'resnet_DAN_full_isbi_train_dropout_patch16_3fc_from55model'
model1_path = '55_model.pth'
model2_path = '55_model.pth'
options['test_folder'] = os.path.join(datasets_path, 'ISBI/train/')


In [ ]:
options['experiment'] = 'resnet_DAN_full_isbi_train_dropout_patch16_fc3'
model1_path='55_model.pth'
model2_path = '55_model.pth'
options['test_folder'] = os.path.join(datasets_path, 'ISBI/train/')

In [26]:
options['experiment'] = 'resnet_DAN_full_isbi_train_2dropout_patch16_1fc'
model1_path='86_model.pth'
model2_path = '80_model.pth'
options['test_folder'] = os.path.join(datasets_path, 'ISBI/train/')


In [24]:
options['experiment']

'resnet_DAN_full_isbi_train_2dropout_patch16_1fc'

In [3]:
options['experiment'] = 'resnet_DAN_full_isbi_train_2dropout_patch16_1fc_frommodel1'
model1_path='86_model.pth'
model2_path = '90_model.pth'
options['test_folder'] = os.path.join(datasets_path, 'ISBI/train/')

In [4]:
model1 = models.DANNet(num_classes=2)
model2 = models.DANNet(num_classes=2)
#load model weights
FILE1 = os.path.join(options['weight_paths'],options['experiment'],'1',model1_path)
FILE2 = os.path.join(options['weight_paths'],options['experiment'],'2',model2_path)


In [5]:
model1 = torch.load(FILE1)
model2 = torch.load(FILE2)
model1.eval()
model2.eval()
correct = 0
print(model1)

DANNet(
  (sharedNet): ResNet(
    (conv1): Conv3d(2, 64, kernel_size=(5, 5, 5), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (down

In [6]:
#change root path
df=pd.read_csv(options['train_csv_path'])
for i in range(len(df)):
    folder = (df.loc[i,'root_path']).split('DataSets')[1]
    path = datasets_path+folder
    df.at[i,'root_path'] = path 
    
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)
df.to_csv(options['train_csv_path'], index=False)

In [7]:
# list scan
fold = 0
# fold train data
df = pd.read_csv(options['train_csv_path'])
# select training scans
train_files = df.loc[df['fold'] != fold, ['patient_id','study']].values
valid_files = df.loc[df['fold'] == fold, ['patient_id', 'study']].values
train_scan_list = [f[0]+f[1] for f in train_files]
valid_scan_list = [f[0]+f[1] for f in valid_files]

_, test_scan_list, _ = next(os.walk(options['test_folder']))

valid_scan_list.sort()
train_scan_list.sort()
test_scan_list.sort()

valid_x_data = {f: {m: os.path.join(options['train_folder'], f, options['tmp_folder'], n)
                    for m, n in zip(options['modalities'], options['preprocess_x_names'])}
                for f in valid_scan_list}
valid_y_data = {f: os.path.join(options['train_folder'], f, options['tmp_folder'],
                                options['preprocess_y_names'][0])
                for f in valid_scan_list}

train_x_data = {f: {m: os.path.join(options['train_folder'], f, options['tmp_folder'], n)
                    for m, n in zip(options['modalities'], options['preprocess_x_names'])}
                for f in train_scan_list}
train_y_data = {f: os.path.join(options['train_folder'], f, options['tmp_folder'],
                                options['preprocess_y_names'][0])
                for f in train_scan_list}
test_data = {f: {m: os.path.join(options['test_folder'], f, options['tmp_folder'], n)
                    for m, n in zip(options['modalities'], options['preprocess_x_names'])}
                for f in test_scan_list}

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
#source_valid_loader = dl.load_training(options, valid_x_data, valid_y_data)
#len_source_valid_dataset = len(source_valid_loader.dataset)
#len_source_valid_loader = len(source_valid_loader)

In [8]:
valid_scan_list

['training01_03',
 'training02_01',
 'training03_04',
 'training04_02',
 'training05_04']

In [9]:
#scan_list=train_scan_list
#x_data = train_x_data

scan_list=valid_scan_list
x_data = valid_x_data
folder = 'train_folder'
#scan_list=test_scan_list
#x_data = test_data
#folder = 'test_folder'

for scan in scan_list:

    total_time = time.time()
    options['tmp_scan'] = scan
    # --------------------------------------------------
    # move things to a tmp folder before starting
    # --------------------------------------------------

    current_folder = os.path.join(options[folder], scan)
    options['tmp_folder'] = os.path.normpath(
        os.path.join(current_folder,  'tmp'))

    # --------------------------------------------------
    # preprocess scans
    # --------------------------------------------------
    #preprocess_scan(current_folder, options)

    # --------------------------------------------------
    # WM MS lesion inference
    # --------------------------------------------------
    seg_time = time.time()

    print("> CNN:", scan, "running WM lesion segmentation")
    sys.stdout.flush()
    options['test_scan'] = scan

    test_x_data = {scan: x_data[scan]}
    out_seg = test_cascaded_model([model1, model2], test_x_data, options, cuda)

    print("> INFO:", scan, "CNN Segmentation time: ",\
        round(time.time() - seg_time), "sec")

    # If input images have been registered before segmentation -> T1w space,
    # then resample the segmentation  back to the original space
    if options['register_modalities']:
        print("> INFO:", scan, "Inverting lesion segmentation masks")
        invert_registration(current_folder, options)

    print("> INFO:", scan, "total pipeline time: ",\
        round(time.time() - total_time), "sec")

    # remove tmps if not set
    if options['save_tmp'] is False:
        try:
            copyfile(os.path.join(current_folder,
                                  options['experiment'],
                                  options['experiment'] +
                                  '_out_CNN.nii.gz'),
                     os.path.join(current_folder,
                                  'out_seg_' +
                                  options['experiment'] +
                                  '.nii.gz'))
            os.rmdir(options['tmp_folder'])
            os.rmdir(os.path.join(options['current_folder'],
                                  options['experiment']))
        except:
            pass

print("> INFO: All processes have been finished. Have a good day!")

> CNN: training01_03 running WM lesion segmentation
> DEBUG  training01_03 Voxels to classify: 1754925


/home/mostafa/Marwa/Code/deep-transfer-learning/UDA/pytorch0.3/DAN/utils/data_load.py:494: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  patches = [new_image[idx] for idx in slices]


> DEBUG: testing current_batch: (859901, 2, 16, 16, 16)
...done!
> DEBUG  training01_03 Voxels to classify: 31038
> DEBUG: testing current_batch: (31038, 2, 16, 16, 16)
...done!
> INFO: training01_03 CNN Segmentation time:  289 sec
> INFO: training01_03 total pipeline time:  289 sec
> CNN: training02_01 running WM lesion segmentation
> DEBUG  training02_01 Voxels to classify: 1379027
> DEBUG: testing current_batch: (427829, 2, 16, 16, 16)
...done!
> DEBUG  training02_01 Voxels to classify: 42246
> DEBUG: testing current_batch: (42246, 2, 16, 16, 16)
...done!
> INFO: training02_01 CNN Segmentation time:  139 sec
> INFO: training02_01 total pipeline time:  139 sec
> CNN: training03_04 running WM lesion segmentation
> DEBUG  training03_04 Voxels to classify: 1779806
> DEBUG: testing current_batch: (807673, 2, 16, 16, 16)
...done!
> DEBUG  training03_04 Voxels to classify: 16804
> DEBUG: testing current_batch: (16804, 2, 16, 16, 16)
...done!
> INFO: training03_04 CNN Segmentation time:  26

In [18]:
options['experiment']

'resnet_DAN_full_isbi_train4'

In [20]:
# change paths for test_data
import nibabel as nib
data_path = os.path.join(options['test_folder'])
new_path = os.path.join('/'.join(options['test_folder'].split('/')[:-2]), options['experiment']+'_seg')
Path(new_path).mkdir(parents=True, exist_ok=True)
_,dirs,_ = next(os.walk(data_path))
for dir_ in dirs:
    file= os.path.join(data_path, dir_, options['experiment'], options['experiment'] + '_hard_seg.nii.gz')
    new_file=os.path.join(new_path, dir_+'_assiut.nii')
    print(file)
    print(new_file)
    if(os.path.isfile(file)):
        img=nib.load(file)
        img.to_filename(new_file)

/home/mostafa/Marwa/DataSets/ISBI/test/test13_02/resnet_DAN_full_isbi_train4/resnet_DAN_full_isbi_train4_hard_seg.nii.gz
/home/mostafa/Marwa/DataSets/ISBI/resnet_DAN_full_isbi_train4_seg/test13_02_assiut.nii
/home/mostafa/Marwa/DataSets/ISBI/test/test10_03/resnet_DAN_full_isbi_train4/resnet_DAN_full_isbi_train4_hard_seg.nii.gz
/home/mostafa/Marwa/DataSets/ISBI/resnet_DAN_full_isbi_train4_seg/test10_03_assiut.nii
/home/mostafa/Marwa/DataSets/ISBI/test/test13_01/resnet_DAN_full_isbi_train4/resnet_DAN_full_isbi_train4_hard_seg.nii.gz
/home/mostafa/Marwa/DataSets/ISBI/resnet_DAN_full_isbi_train4_seg/test13_01_assiut.nii
/home/mostafa/Marwa/DataSets/ISBI/test/test01_01/resnet_DAN_full_isbi_train4/resnet_DAN_full_isbi_train4_hard_seg.nii.gz
/home/mostafa/Marwa/DataSets/ISBI/resnet_DAN_full_isbi_train4_seg/test01_01_assiut.nii
/home/mostafa/Marwa/DataSets/ISBI/test/test11_01/resnet_DAN_full_isbi_train4/resnet_DAN_full_isbi_train4_hard_seg.nii.gz
/home/mostafa/Marwa/DataSets/ISBI/resnet_DAN_fu

/home/mostafa/Marwa/DataSets/ISBI/test/test10_05/resnet_DAN_full_isbi_train4/resnet_DAN_full_isbi_train4_hard_seg.nii.gz
/home/mostafa/Marwa/DataSets/ISBI/resnet_DAN_full_isbi_train4_seg/test10_05_assiut.nii
/home/mostafa/Marwa/DataSets/ISBI/test/test12_02/resnet_DAN_full_isbi_train4/resnet_DAN_full_isbi_train4_hard_seg.nii.gz
/home/mostafa/Marwa/DataSets/ISBI/resnet_DAN_full_isbi_train4_seg/test12_02_assiut.nii
/home/mostafa/Marwa/DataSets/ISBI/test/test02_03/resnet_DAN_full_isbi_train4/resnet_DAN_full_isbi_train4_hard_seg.nii.gz
/home/mostafa/Marwa/DataSets/ISBI/resnet_DAN_full_isbi_train4_seg/test02_03_assiut.nii
/home/mostafa/Marwa/DataSets/ISBI/test/test05_01/resnet_DAN_full_isbi_train4/resnet_DAN_full_isbi_train4_hard_seg.nii.gz
/home/mostafa/Marwa/DataSets/ISBI/resnet_DAN_full_isbi_train4_seg/test05_01_assiut.nii
/home/mostafa/Marwa/DataSets/ISBI/test/test10_01/resnet_DAN_full_isbi_train4/resnet_DAN_full_isbi_train4_hard_seg.nii.gz
/home/mostafa/Marwa/DataSets/ISBI/resnet_DAN_fu

In [13]:
data_path, new_path

('/home/mostafa/Marwa/DataSets/ISBI/test/isbi_full_train_2modalities',
 '/home/mostafa/Marwa/DataSets/ISBI/resnet_full_isbi_train_seg')